In [ ]:
# Tests for methods that merge horizons
import sys
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

sys.path.append('../..') # for running py-script
sys.path.append('../../..') # for running this notebook directly

from seismiqb import Field, Horizon, plot_image

In [ ]:
# CUBE_PATH = 'path/to/cube'
# HORIZON_PATH = 'path/to/horizon'

# FIGSIZE = (12, 7)
# SHOW_FIGURES = False # Whether to show additional figures

In [ ]:
def test_horizons_equality(horizon_1, horizon_2, test_name):
    " Check two horizons equality."
    assert np.array_equal(horizon_1.full_matrix, horizon_2.full_matrix), f"{test_name} test failed"
    assert np.array_equal(horizon_1.points, horizon_2.points), f"{test_name} test failed"

In [ ]:
%%time
field = Field(CUBE_PATH)

horizon = Horizon(HORIZON_PATH, field=field)

horizon.show(show=SHOW_FIGURES)

# Merge two horizons

In [ ]:
configs = [
#   (first horizon border, second horizon border, verify_merge code)
    (len(horizon.points)//2, len(horizon.points)//2-len(horizon.points)//10, 3),        # overlap
    (len(horizon.points)//10, len(horizon.points)//10, 2),                              # close without overlap
    (len(horizon.points)//10, len(horizon.points)//2, 1),                               # spatially_far
    (len(horizon.points)//10, len(horizon.points)//10-len(horizon.points)//20, 0)       # height-wise far
]

In [ ]:
%%time
# Horizon.*_merge methods

for (border_1, border_2, code) in configs:
    # Make new horizons
    horizon_1 = Horizon(horizon.points[:border_1, :],
                        field=field, name='horizon_1')

    horizon_2 = Horizon(horizon.points[border_2:, :],
                        field=field, name='horizon_2')

    if code == 0: # make horizons height-wise far
        horizon_2.points[:, 2] += 50

    # Check merge code
    assert horizon_1.verify_merge(horizon_2, adjacency=1)[0] == code, f"merge two horizons with code {code} test failed"

    # Merge and check horizons if they are mergeable
    if code == 3:
        merged_horizon = Horizon.overlap_merge(horizon_1, horizon_2)

        test_horizons_equality(horizon, merged_horizon, test_name="merge overlapped horizons")

    elif code == 2:
        merged_horizon = horizon_1.adjacent_merge(horizon_2)

        test_horizons_equality(horizon, merged_horizon, test_name="merge close without overlap horizons")
    
    print(f"Merge code {code} was successfully tested.")

In [ ]:
%%time
# average_horizons method

for (border_1, border_2, code) in configs:
    # Make new horizons
    horizon_1 = Horizon(horizon.points[:border_1, :],
                        field=field, name='horizon_1')

    horizon_2 = Horizon(horizon.points[border_2:, :],
                        field=field, name='horizon_2')

    if code == 0: # make horizons height-wise far
        horizon_2.points[:, 2] += 50

    # Get averaged horizons points
    averaged_horizon_info = Horizon.average_horizons([horizon_1, horizon_2])

    averaged_horizon_points = Horizon.matrix_to_points(
        averaged_horizon_info[1]['matrix']
    ).astype(int)

    # Calculate averaged horizons points with horizons points
    points_1 = pd.DataFrame(horizon_1.points, columns=['i','x','h']).set_index(['i', 'x'])
    points_2 = pd.DataFrame(horizon_2.points, columns=['i','x','h']).set_index(['i', 'x'])

    # Overlapping part
    intersection_idxs = points_1.index.intersection(points_2.index)

    intersection_average = (points_1.loc[intersection_idxs] + points_2.loc[intersection_idxs]) // 2
    intersection_average = intersection_average.astype(int)

    # Not-overlapping part
    non_intersected_points_1 = points_1.drop(intersection_idxs)
    non_intersected_points_2 = points_2.drop(intersection_idxs)

    # Get averaged points
    averaged_points = pd.concat(
        [non_intersected_points_1, non_intersected_points_2, intersection_average]
    )

    averaged_points.sort_index(inplace=True)
    averaged_points = averaged_points.reset_index().to_numpy()

    assert np.array_equal(
        averaged_horizon_points, averaged_points
    ), f"average_horizons for two horizons with code {code} test failed"

    print(f"Merge code {code} was successfully tested.")

# Merge multiple horizons

We split a horizon in multiple parts, merge them into one horizon and compare it with original one.

In [ ]:
horizon.filter()

zero_traces = np.tril(horizon.full_matrix).astype(bool)
horizon.filter(zero_traces)

horizon.filter_disconnected_regions()

horizon.show(show=SHOW_FIGURES, figsize=FIGSIZE)

In [ ]:
def split_and_merge_horizon_test(horizon, crop_shape, overlap, adjacency, minsize=1):
    """ Split a horizon in crops with overlap and merge them.
    
    Merged horizon compared with the original.
    
    Parameters
    ----------
    horizon : instance of the `Horizon` class
        Seismic horizon.
    crop_shape : sequence of two integers
        Crop shape (ilines, xlines).
    overlap : sequence of two integers
        Crops overlap by (ilines, xlines).
    adjacency : int
        Margin to consider horizons to be close (spatially).
    minsize : int
        Minimum length of a horizon to be saved in merging.
    """

    borders = []

    # Get  splitting borders
    for axis in range(2):
        crop_num_ = np.ceil(horizon.full_matrix.shape[axis]/(crop_shape[axis] - overlap[axis])).astype(int)
        axis_borders = [(i*(crop_shape[axis] - overlap[axis]), (i*(crop_shape[axis] - overlap[axis]) + crop_shape[axis])) for i in range(crop_num_)]

        borders.append(axis_borders)

    horizons = []

    # Split a horizon
    for i_line_border_num in range(0, len(borders[0])):
        for x_line_border_num in range(0, len(borders[1])):
            horizon_i_borders = borders[0][i_line_border_num]
            horizon_x_borders = borders[1][x_line_border_num]
            
            cutted_horizon_matrix = horizon.full_matrix[horizon_i_borders[0]:horizon_i_borders[1],
                                                        horizon_x_borders[0]:horizon_x_borders[1]]

            if not np.all(cutted_horizon_matrix == horizon.FILL_VALUE):
                horizon_ = Horizon(cutted_horizon_matrix,
                                  i_min=horizon_i_borders[0], x_min=horizon_x_borders[0],
                                  field=field, name=f'horizon_{i_line_border_num*x_line_border_num}')

                horizons.append(horizon_)

    # Horizon merge
    merged_horizon = Horizon.merge_list(horizons, adjacency=adjacency, minsize=minsize)[0]

    params = f"crop_shape: {crop_shape}, overlap: {overlap}, adjacency: {adjacency}"
    
    # Take a look at horizon parts summation
    if SHOW_FIGURES:
        merged_ = np.zeros_like(horizons[0].full_matrix)
        for h in horizons:
            merged_ += (h.full_matrix > 0).astype(int)
        plot_image([merged_, merged_horizon.full_matrix], separate=True,
                   figsize=FIGSIZE, title=params, fontsize=14)

    if (overlap[0] >= 0) and (overlap[1] >= 0):
        test_horizons_equality(
            horizon_1=horizon, horizon_2=merged_horizon,
            test_name=f"""merge list ({params})"""
        )

    else:
        # Approximate calculation of empty traces coverage on horizon
        width = np.abs(overlap)
        
        n_ilines = horizon.shape[0] // crop_shape[0]
        n_xlines = horizon.shape[1] // crop_shape[1]
        
        empty_traces = n_ilines * width[0] * horizon.i_length + n_xlines * width[1] * horizon.x_length
        
        intersections = width[0] * width[1] * n_ilines * n_xlines
        empty_traces -= intersections
        
        coverage_empty_traces = empty_traces/horizon.size
        
        assert coverage_empty_traces >= (horizon.coverage - merged_horizon.coverage)

        assert np.array_equal(
            horizon.full_matrix[merged_horizon.presence_matrix],
            merged_horizon.full_matrix[merged_horizon.presence_matrix]
        )

In [ ]:
overlaps = [(0, 0), (2, 2), (-1, -1), (1, -2), (0, 1)]

adjacency = np.max(np.abs(overlaps)) + 1

crop_shapes = [
    # Split horizon into squared crops
    (horizon.full_matrix.shape[0]//10, horizon.full_matrix.shape[1]//10),
    # Split horizon by i_lines
    (horizon.full_matrix.shape[0], horizon.full_matrix.shape[1]//20),
    # Split horizon by x_lines
    (horizon.full_matrix.shape[0]//20, horizon.full_matrix.shape[1])
]

for crop_shape in crop_shapes:
    for overlap in overlaps:
        split_and_merge_horizon_test(horizon=horizon, crop_shape=crop_shape,
                                     overlap=overlap, adjacency=adjacency,
                                     minsize=0)